A notebook for making a 100-frame movie from 3D dust for Linnea as a test

Two environment variables need to be set in bash: DROPBOX_ROOT, and FFMPEG_ROOT. The DROPBOX_ROOT folder is the path to the top level directory of the dust-holgrams folder shared by Josh. E.g. in bash,

export DROPBOX_ROOT='/Users/catherinezucker/Dropbox/dust-holograms/'

You can also set the path to the FFMPEG executable:

export FFMPEG_PATH='/usr/local/bin'

Ideally, you would set these in your bash_profile profile  (now called zprofile in the latest OS operating systems). Alternatively, if you want to set one of these variables locally in this notebook, you can use:

```
import os
os.environ['DROPBOX_ROOT'] = '/Users/catherinezucker/Dropbox/dust-holograms'
os.environ['FFMPEG_ROOT'] = '/Usr/local/bin'

```

First thing we need to do is figure out how to write some json

In [1]:
import json
import numpy as np
import os
%matplotlib inline
#%matplotlib notebook
from matplotlib import pyplot as plt
from astropy import table

Josh is having some trouble with ```.zprofile``` so:

In [2]:
if (os.environ.keys() != 'DROPBOX_HOME'):
    os.environ['DROPBOX_ROOT'] = '/Users/jegpeek/Dropbox/dust-holograms'
    os.environ['FFMPEG_ROOT'] = '/Users/jegpeek/'

Using a variable called ```run_name``` to record everything we need about the run

In [3]:
def define_paths(run_name):
    if os.path.isdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name)) == False:
        os.mkdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name))
        os.mkdir("{}/{}/frames/".format(os.environ['DROPBOX_ROOT'],run_name))

This is the camera properties, which we will fix.

In [4]:
arc = False
if arc:
    ec = "szyz"
if not arc:
    ec = "rxyz"

t = True
cprops ={
    "projection": "stereographic",
    "step_size": 1,
    "max_dist": 500.0,
    "fov": 45.0,
    "x_pix": 60,
    "y_pix": 40,
    "vmax": "auto",
    "clip_mode": "tanh",
    "fuzzy": t,
    "randomize_angles": t,
    "euler_convention": ec}

In [5]:
print(cprops)

{'projection': 'stereographic', 'step_size': 1, 'max_dist': 500.0, 'fov': 45.0, 'x_pix': 60, 'y_pix': 40, 'vmax': 'auto', 'clip_mode': 'tanh', 'fuzzy': True, 'randomize_angles': True, 'euler_convention': 'rxyz'}


_In arc mode_ Linnea asked for 100 frames orbiting around a point with equal distance. We'll assume that the middle of these 100 frames is the Sun, and we'll set some sweep out angle in the Galactic Plane.

In [6]:
nframes = 10 # 100 frames
if arc:
    angle_sweep = 45 # half of this CCW and half CW from the sun

In [7]:
def sweep(xc, yc, angle_sweep, nframes):
    R = np.sqrt(xc**2+ yc**2)
    phi =np.arctan2(yc, xc)*180/np.pi
    dangs = np.linspace(0, angle_sweep, nframes)
    xs = xc - R*np.cos((phi-angle_sweep/2+dangs)*np.pi/180)
    ys = yc - R*np.sin((phi-angle_sweep/2+dangs)*np.pi/180)
    angs = (dangs-angle_sweep/2+phi)*np.pi/180 ## wait are these in RADIANs??
    return xs, ys, angs

In [8]:
def plot_ang(x0, y0, xs, ys, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(x0, y0)
    plt.scatter(0, 0, c='red')
    plt.plot(xs, ys)
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

In track mode we'll move from one point to another while pointing at the cloud and keeping to top of the camera perpindicular to the plane that contains the track and cloud

Given the current position of the viewer, the unit direction toward the start of the track, and the cloud position, compute the angles of rotation

In [9]:
def determine_angles(v_pos, c_pos, t_hat):
    x_hat = np.array([1, 0, 0])
    y_hat = np.array([0, 1, 0])
    z_hat = np.array([0, 0, 1])
    # the vector to the cloud
    c_vec = c_pos-v_pos 
    # the unit vector toward the cloud
    c_hat = c_vec/np.sqrt(np.sum(c_vec**2))
    
    #FIRST
    # the unit vector perp to the plane containing the track and the cloud
    n_OCT = np.cross(t_hat, c_hat) #CHECK ORDER (checked)
    n_hat_OCT = n_OCT/np.sqrt(np.sum(n_OCT**2))
    print(n_OCT)
    # find the new orientation of the top of the camera
    v_y_prime = np.cross(n_hat_OCT, x_hat) #CHECK ORDER (checked)
    v_hat_y_prime = v_y_prime/np.sqrt(np.sum(v_y_prime**2))
    # and find the angle to rotate first
    theta = np.arccos(np.dot(v_hat_y_prime,y_hat))
    #print(theta)
    
    #SECOND
    # the unit vector perp to the plane that contains the rotation 
    n_phi = np.cross(x_hat,v_hat_y_prime) #CHECK ORDER (checked)
    n_hat_phi = n_phi/np.sqrt(np.sum(n_phi**2))
    # the vector being rotated to
    v_x_prime_prime = np.cross(n_hat_OCT, n_hat_phi) #CHECK ORDER (checked)
    v_hat_x_prime_prime = v_x_prime_prime/np.sqrt(np.sum(v_x_prime_prime**2))
    # and the angle rotated
    phi = np.arccos(np.dot(v_hat_x_prime_prime, x_hat))
    
    #FINALLY
    zeta = np.arccos(np.dot(v_hat_x_prime_prime, c_hat))
    
    return theta, phi, zeta
    

given some inputs build a track

In [10]:
def track(xt1, yt1, zt1, xt2, yt2, zt2, nframes):
    xts = np.linspace(xt1, xt2, nframes)
    yts = np.linspace(yt1, yt2, nframes)
    zts = np.linspace(zt1, zt2, nframes)
    t_vec = np.array([xt2-xt1, yt2-yt1, zt2-zt1])
    t_hat = t_vec/np.sqrt(np.sum(t_vec**2))
    return xts, yts, zts, t_hat

In [11]:
def build_fprops(fprops, cprops, angs, xs, ys, zc):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zc],
          "angles": [angs[i], 0.0, 0.0]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

In [12]:
def build_fprops_track(fprops, cprops, theatas, phis, zetas, xs, ys, zs):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zs[i]],
          "angles": [thetas[i], phis[i], zetas[i]]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

Let's read in a list of molecular clouds and make movies for each one:

In [13]:
clouds = table.Table.read('{}/Holo_Cloud_Targets.csv'.format(os.environ['DROPBOX_ROOT']))

In [14]:
for c in clouds[0:4]:
    print(c['cloud'])
    run_name = c['cloud'] + '_track_test_1'
    define_paths(run_name)
    if arc:
        xs, ys, angs = sweep(c['x'], c['y'], angle_sweep, nframes)
        plot_ang(c['x'], c['y'], xs, ys, run_name)
        # we make an empty list of frames to which we can append frames
        fprops = []
        allprops = build_fprops(fprops, cprops, angs, xs, ys, np.float(c['z']))
    if not arc:
        #hardcoding a track
        xts, yts, zts, t_hat = track(2, 5, 7, -2, -5, -7, nframes)
        fprops = []
        thetas = np.zeros(nframes)
        phis = np.zeros(nframes)
        zetas = np.zeros(nframes)
        for j in np.arange(nframes):
            v_pos = np.array([xts[j], yts[j], zts[j]])
            c_pos = np.array([c['x'], c['y'], c['z']])
            thetas[j], phis[j], zetas[j] = determine_angles(v_pos, c_pos, t_hat)
        allprops = build_fprops_track(fprops, cprops, thetas, phis, zetas, xts, yts, zts)

    with open('{}/{}/{}.json'.format(os.environ['DROPBOX_ROOT'],run_name,run_name), 'w') as outfile:
        json.dump(allprops, outfile,indent=2)
    os.system("python3 project_frames.py {}/{}/{}.json {}/leike2020_bayestar19_splice.npy {}/{}/frames/{}_{{:05d}}.png"
          .format(os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],os.environ['DROPBOX_ROOT'],run_name, run_name))
    
    os.system("{}/ffmpeg -r 30 -start_number 0 -i {}/{}/frames/{}_%05d.png -c:v libx264 -s 600x400 -r 30 -pix_fmt yuv420p {}/{}/{}.mp4"
          .format(os.environ['FFMPEG_ROOT'],os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],run_name, run_name))


Chamaeleon
[-0.49616934 -0.42002155  0.44177806]
[-0.49938651 -0.42274498  0.44464256]
[-0.50261048 -0.42547416  0.44751311]
[-0.50584001 -0.42820805  0.45038861]
[-0.50907379 -0.43094553  0.45326789]
[-0.51231044 -0.43368545  0.45614974]
[-0.51554854 -0.43642659  0.45903286]
[-0.51878657 -0.43916768  0.46191594]
[-0.52202298 -0.44190739  0.46479756]
[-0.52525611 -0.44464433  0.46767627]
Loaded specifications for 10 images.


100% (10 of 10) |########################| Elapsed Time: 0:00:02 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.0006002974811344756
frame 1: vmax = 0.0005883440284233075
frame 2: vmax = 0.0005781417014077306
frame 3: vmax = 0.0006118517606591923
frame 4: vmax = 0.0005808474135628785
frame 5: vmax = 0.0006096765809088538
frame 6: vmax = 0.0005758763246230956
frame 7: vmax = 0.0006976815792259003
frame 8: vmax = 0.0009560700988422468
frame 9: vmax = 0.0009012382623623125
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Ophiuchus
[-0.22761385 -0.70860914  0.57118191]
[-0.22642502 -0.70490807  0.56819863]
[-0.22520574 -0.70111221  0.56513894]
[-0.22395788 -0.69722736  0.56200751]
[-0.22268327 -0.69325925  0.55880897]
[-0.22138375 -0.68921357  0.55554791]
[-0.22006111 -0.68509592  0.55222883]
[-0.21871712 -0.68091179  0.54885617]
[-0.21735351 -0.6766666   0.54543429]
[-0.21597198 -0.67236561  0.54196743]
Loaded specifications for 10 images.


...

ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Taurus
[ 0.24105703  0.68261944 -0.55645875]
[ 0.24232603  0.68621297 -0.55938813]
[ 0.24357315  0.68974452 -0.56226698]
[ 0.24479693  0.69320999 -0.56509197]
[ 0.24599593  0.69660529 -0.56785976]
[ 0.24716868  0.69992626 -0.57056695]
[ 0.24831371  0.70316872 -0.57321015]
[ 0.24942954  0.70632851 -0.57578595]
[ 0.2505147   0.70940143 -0.57829094]
[ 0.25156771  0.71238332 -0.58072172]
Loaded specifications for 10 images.


100% (10 of 10) |########################| Elapsed Time: 0:00:02 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.0024981274587189545
frame 1: vmax = 0.0017329092789786955
frame 2: vmax = 0.0017319503609032837
frame 3: vmax = 0.0015819420011757756
frame 4: vmax = 0.0017589842606867024
frame 5: vmax = 0.0017822166530168033
frame 6: vmax = 0.0015627234507410322
frame 7: vmax = 0.0015290174745678088
frame 8: vmax = 0.0014800350137047645
frame 9: vmax = 0.0013439259067781677
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay